<img src="https://raw.githubusercontent.com/astrolabsoftware/fink-notebook-template/main/Fink_PrimaryLogo_WEB.jpg" width=400 />

# Fink case study: name resolver

## Goal

This notebook shows how to resolve object names using various databases.

## What is behind?

Giving names is a complex process, and often objects end up with multiple names or designations. At the era of big data, this is especially harder as the need to names million of objects quickly lead to non-intuitive designation process. We will not come up with a new naming scheme here, but rather offer a new service to explore existing names for a given object. Currently, you will be able to resolve ZTF object names in SIMBAD, the Transient Name Server, and Solar System databases known by the Quaero service from SSODNET -- and vice versa meaning given an object in one of those 3 databases, you will know if there is a corresponding ZTF objects. This service is expanding, and we will keep adding new source of information.

In [1]:
import io
import requests
import pandas as pd

## TNS to ZTF

I have a TNS identifier, are there ZTF objects corresponding?

In [2]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'tns',
    'name': 'SN 2023vwy'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:declination,d:fullname,d:internalname,d:ra,d:type,key:time
0,52.219894,SN 2023vwy,ATLAS23url,47.765951,SN Ia,1701889538816
1,52.219894,SN 2023vwy,ZTF23abmwsrw,47.765951,SN Ia,1701889538816


## ZTF to TNS

I have a ZTF object name, are there counterparts in TNS?

In [3]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'tns',
    'reverse': True,
    'name': 'ZTF23abmwsrw'
  }
)

# Format output in a DataFrame
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:declination,d:fullname,d:internalname,d:ra,d:type,key:time
0,52.219904,AT 2023vwy,ZTF23abmwsrw,47.765935,nan,1698788550829
1,52.219894,SN 2023vwy,ZTF23abmwsrw,47.765951,SN Ia,1701889538816


## SIMBAD to ZTF

I have an astronomical object name referenced in SIMBAD, are there counterparts in ZTF? As these objects can be extended, we typically provide coordinates, and then you need to run a conesearch:

In [4]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'simbad',
    'name': 'Markarian 2'
  }
)

if r.json() != []:
    print('Object found!')
    print(r.json())
    print()

    r = requests.post(
      'https://fink-portal.org/api/v1/explorer',
      json={
        'ra': r.json()[0]['jradeg'],
        'dec': r.json()[0]['jdedeg'],
        'radius': 60
      }
    )
    
    # Format output in a DataFrame
    pdf = pd.read_json(io.BytesIO(r.content))
    print('Object(s) in ZTF: {}'.format(pdf['i:objectId'].values))
else:
    print('No objects found')

Object found!
[{'name': 'Si=Simbad, all IDs (via url)', 'oid': 1579005, 'oname': 'Mrk 2', 'otype': 'GiG', 'jpos': '01:54:53.80 +36:55:04.6', 'jradeg': 28.7241958, 'jdedeg': 36.9179556, 'refPos': '2006AJ....131.1163S', 'z': None, 'MType': 'SBa', 'nrefs': 138}]

Object(s) in ZTF: ['ZTF18aabfjoi']


## ZTF to SIMBAD

I have a ZTF object name, are there counterparts in SIMBAD?

In [5]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'simbad',
    'reverse': True,
    'name': 'ZTF18aabfjoi'
  }
)
pdf = pd.read_json(io.BytesIO(r.content))
pdf

,d:cdsxmatch,i:dec,i:objectId,i:ra
0,GinGroup,36.917909,ZTF18aabfjoi,28.724092
1,GinGroup,36.917912,ZTF18aabfjoi,28.724130
2,GinGroup,36.917924,ZTF18aabfjoi,28.724110
3,GinGroup,36.917913,ZTF18aabfjoi,28.724100
4,GinGroup,36.917892,ZTF18aabfjoi,28.724094


## SSO to ZTF

I have a SSO name or number, are there ZTF objects corresponding? Note that by default the search has a `contain` strategy -- namely we will look for all objects whose name _contains_ the provided search term (this is not a exact name search). By default, we only returns the first 10 matches. We will add in the future an option to specify exact search as well.

In [6]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'ssodnet',
    'name': '1999 VK210'
  }
)

pdf = pd.read_json(io.BytesIO(r.content))
pdf

,type,system,class,updated,ephemeris,id,aliases,links,physical-ephemeris,name,parent
0,Asteroid,Sun,"[MB, Inner]",2023-07-24,True,Julienpeloton,"[1976 SU10, 1998 HC30, 1999 VK210, 2001 DR108,...",{'self': 'https://api.ssodnet.imcce.fr/quaero/...,False,Julienpeloton,Sun


In [7]:
print(pdf['aliases'].values)

[list(['1976 SU10', '1998 HC30', '1999 VK210', '2001 DR108', '2001 FA193', '2001 FY172', '2033803', '33803', 'J76S10U', 'J98H30C', 'J99VL0K', 'K01DA8R', 'K01FH2Y', 'K01FJ3A'])]


In [8]:
for name in pdf['aliases'].values[0]:
    r = requests.post(
      'https://fink-portal.org/api/v1/sso',
      json={
        'n_or_d': name,
      }
    )

    if r.json() != []:
        print('{} found in ZTF with {} alerts'.format(name, len(r.json())))        

33803 found in ZTF with 92 alerts


## ZTF to SSO

I have a ZTF object name, is there a counterpart in the SsODNet quaero database, and what are all the aliases?

In [9]:
r = requests.post(
  'https://fink-portal.org/api/v1/resolver',
  json={
    'resolver': 'ssodnet',
    'reverse': True,
    'name': 'ZTF23abpkzcn'
  }
)

if r.json() != []:
    name = r.json()[0]['i:ssnamenr']
    print('Asteroid counterpart found with designation {}'.format(name))
    print()

    r = requests.post(
      'https://fink-portal.org/api/v1/resolver',
      json={
        'resolver': 'ssodnet',
        'name': name
      }
    )
    
    pdf = pd.read_json(io.BytesIO(r.content))
    print('In the quaero database, this corresponds to: ')
    print(pdf[['type', 'name', 'class', 'aliases']])

Asteroid counterpart found with designation 8467

In the quaero database, this corresponds to: 
         type         name                    class   
0  Spacecraft     OPS 4428  [LEO, Payload, Decayed]  \
1    Asteroid  Benoitcarry              [MB, Outer]   
2    Asteroid    2002 VG84             [MB, Middle]   
3    Asteroid    2002 VW87              [MB, Outer]   
4    Asteroid    2002 VT88             [MB, Middle]   
5    Asteroid    2002 VW88             [MB, Middle]   
6    Asteroid    2002 VT87             [MB, Middle]   
7    Asteroid    2002 VS83             [MB, Middle]   
8    Asteroid    2002 VM88             [MB, Middle]   
9    Asteroid    2002 VF83             [MB, Middle]   

                                          aliases  
0                               [1975-114A, 8467]  
1      [08467, 1981 ES35, 2008467, 8467, J81E35S]  
2                       [2084672, 84672, K02V84G]  
3                       [2084675, 84675, K02V87W]  
4                       [2084677, 8467